In [ ]:
import numpy as np
from cloth_tools.stations.competition_station import CompetitionStation
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_dual_ur5e_and_table_to_builder
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.path.execution import execute_dual_arm_joint_path
from cloth_tools.drake.visualization import publish_dual_arm_joint_path
from cloth_tools.path.execution import calculate_dual_path_duration

In [ ]:
# Only run this cell once as this accesses the real hardware
station = CompetitionStation()
dual_arm = station.dual_arm

In [ ]:
current_joints_left = dual_arm.left_manipulator.get_joint_configuration()
current_joints_right = dual_arm.right_manipulator.get_joint_configuration()
current_joints = np.concatenate((current_joints_left, current_joints_right))

with np.printoptions(precision=2, suppress=True):
    print("Current joints: ", current_joints)

In [ ]:
# Creating the default scene
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_indices, gripper_indices = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)
diagram, context = finish_build(robot_diagram_builder, meshcat)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[*arm_indices, *gripper_indices],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

is_state_valid_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

arm_left_index, arm_right_index = arm_indices
plant.SetPositions(plant_context, arm_left_index, current_joints_left)
plant.SetPositions(plant_context, arm_right_index, current_joints_right)
diagram.ForcedPublish(context)

In [ ]:
collision_checker.CheckConfigCollisionFree(current_joints)

In [ ]:
home_joints_left = np.deg2rad([180, -135, 95, -50, -90, -90])
home_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])

home_joints = np.concatenate((home_joints_left, home_joints_right))
collision_checker.CheckConfigCollisionFree(home_joints)

In [ ]:
planner = DualArmOmplPlanner(is_state_valid_fn)

In [ ]:
current_joints_left = dual_arm.left_manipulator.get_joint_configuration()
current_joints_right = dual_arm.right_manipulator.get_joint_configuration()
current_joints = np.concatenate((current_joints_left, current_joints_right))

path = planner.plan_to_joint_configuration(
    current_joints_left, current_joints_right, home_joints_left, home_joints_right
)
duration = calculate_dual_path_duration(path)
publish_dual_arm_joint_path(path, duration, meshcat, diagram, context, *arm_indices)

In [ ]:
import matplotlib.pyplot as plt

# create two vertically stacked wide subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20, 6))

axes[0].set_title("Left Arm Joint Positions")
axes[1].set_title("Right Arm Joint Positions")

# set xlim to 0 -> len(path) -1
axes[0].set_xlim(0, len(path) - 1)
axes[1].set_xlim(0, len(path) - 1)

# set ylims to -2pi to 2pi
axes[0].set_ylim(-2 * np.pi, 2 * np.pi)
axes[1].set_ylim(-2 * np.pi, 2 * np.pi)

path_left, path_right = zip(*path)

for i in range(6):
    path_left_joint_i = [joints[i] for joints in path_left]
    path_right_joint_i = [joints[i] for joints in path_right]
    axes[0].plot(path_left_joint_i)
    axes[1].plot(path_right_joint_i)

In [ ]:
execute_dual_arm_joint_path(dual_arm, path)

In [ ]:
scenario4_start_joint_left = np.deg2rad([0, -90, -90, -90, 90, 0])
scenario4_start_joint_right = np.deg2rad([-136, -116, -110, -133, 40, 0])

path_to_scenario4 = planner.plan_to_joint_configuration(
    home_joints_left, home_joints_right, scenario4_start_joint_left, scenario4_start_joint_right
)

duration = calculate_dual_path_duration(path_to_scenario4)
publish_dual_arm_joint_path(path_to_scenario4, duration, meshcat, diagram, context, *arm_indices)

In [ ]:
execute_dual_arm_joint_path(dual_arm, path_to_scenario4)